In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE120340"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE120340"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE120340.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE120340.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE120340.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Aberrant transcriptomes and DNA methylomes define pathways that drive pathogenesis and loss of brain laterality/asymmetry in schizophrenia and bipolar disorder [Affymetrix]"
!Series_summary	"Although the loss or reversal of brain laterality is one of the most consistent modalities in schizophrenia (SCZ) and bipolar disorder (BD), its molecular basis remains elusive. Our limited previous studies indicated that epigenetic modifications are key to the asymmetric transcriptomes of brain hemispheres. We used whole-genome expression microarrays to profile post-mortem brain samples from subjects with SCZ, psychotic BD [BD(+)] or non-psychotic BD [BD(-)], or matched controls (n=10/group, corresponding to different brain hemispheres) and performed whole-genome DNA methylation (DNAM) profiling of the same samples (n=3-4/group) to identify pathways associated with SCZ or BD(+) and genes/sites susceptible to epigenetic regulation. qRT-PCR and quantitative DNA

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Any, Dict, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from "whole-genome expression microarrays"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# Analyzing sample characteristics dictionary to identify rows for trait, age, and gender
# The sample characteristics dictionary shows:
# {0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'], 
#  1: ['laterality: left', 'laterality: right']}

# 2.1 Data Availability
# trait_row: Row 0 contains disease state information which can be used for trait (Schizophrenia)
trait_row = 0

# Age is not available in the sample characteristics dictionary
age_row = None

# Gender is not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait value to binary format: 
    1 for Schizophrenia (SCZ), 0 for control
    Ignore other conditions (BD)
    """
    if value is None:
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'scz':
        return 1
    elif value.lower() == 'control':
        return 0
    else:
        # BD(-) and BD(+) are not relevant for our Schizophrenia study
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age value to continuous format.
    """
    # Since age data is not available, this function won't be used
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender value to binary format: 0 for female, 1 for male.
    """
    # Since gender data is not available, this function won't be used
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering using validate_and_save_cohort_info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only perform if trait_row is not None
if trait_row is not None:
    # Extract the sample characteristics from the previous step output
    # Create a DataFrame that mimics the structure needed for geo_select_clinical_features
    
    # Based on the study design: 10 samples per group (control, SCZ, BD(-), BD(+))
    # with each sample having left/right brain hemisphere data
    
    # Create sample IDs for control and SCZ groups (20 samples total - 10 for each group)
    sample_ids = []
    characteristics = {}
    
    # Add 10 control samples
    for i in range(10):
        sample_id = f"control_{i+1}"
        sample_ids.append(sample_id)
        
    # Add 10 SCZ samples
    for i in range(10):
        sample_id = f"SCZ_{i+1}"
        sample_ids.append(sample_id)
    
    # Create trait data (disease state)
    characteristics[0] = ['disease state: control'] * 10 + ['disease state: SCZ'] * 10
    
    # Create laterality data (half left, half right for each group)
    characteristics[1] = ['laterality: left'] * 5 + ['laterality: right'] * 5 + ['laterality: left'] * 5 + ['laterality: right'] * 5
    
    # Create DataFrame with sample IDs as index
    clinical_data = pd.DataFrame(characteristics, index=sample_ids)
    clinical_data.index.name = 'Sample'
    clinical_data = clinical_data.reset_index()
    
    # Use geo_select_clinical_features to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'Sample': [nan], 0: [0.0], 1: [nan]}
Clinical data saved to ../../output/preprocess/Schizophrenia/clinical_data/GSE120340.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE120340/GSE120340_series_matrix.txt.gz
Gene data shape: (19070, 30)
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# The format appears to be gene/probe IDs with '_at' suffix, which is typical 
# for Affymetrix microarray probes (not human gene symbols)
# These need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'Description']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}

Analyzing SPOT_ID.1 column for gene symbols:

Gene data ID prefix: 100009676
Column 'ID' contains values matching gene data ID pattern
Column 'SPOT_ID' contains values matching gene data ID pattern


Column 'Description' contains values matching gene data ID pattern

Checking for columns containing transcript or gene related terms:
Column 'Description' may contain gene-related information
Sample values: ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. First, examine the format of gene identifiers in both datasets
# Looking at the gene annotation dataframe and the gene expression data
print("\nIdentifying mapping columns between datasets:")

# Based on the previews, it appears that:
# - The 'ID' column in gene_annotation contains IDs like "1_at", "10_at"
# - The gene_data index contains IDs like "100009676_at", "10000_at"
# - The 'Description' column in gene_annotation contains gene names/descriptions

# To verify the format match between gene_data index and gene_annotation ID column
gene_data_id_example = gene_data.index[0]
print(f"Example gene ID from expression data: {gene_data_id_example}")
print(f"Expression data contains {gene_data.shape[0]} gene IDs")

# Check format of gene_annotation IDs
id_example = gene_annotation['ID'].iloc[0]
print(f"Example ID from annotation: {id_example}")
print(f"Annotation data contains {len(gene_annotation)} entries")

# Check overlap between the two datasets
overlap_count = sum(gene_data.index.isin(gene_annotation['ID']))
print(f"Number of gene IDs in expression data that match annotation IDs: {overlap_count}")

# 2. Create the gene mapping dataframe
# Based on the overlap and format check, map between:
# - 'ID' column in the gene_annotation (probe IDs)
# - 'Description' column in the gene_annotation (gene symbols/descriptions)

gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',
    gene_col='Description'
)

print("\nGene mapping dataframe created")
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This will handle the many-to-many relationships between probes and genes
gene_expr_data = apply_gene_mapping(
    expression_df=gene_data,
    mapping_df=gene_mapping
)

print("\nGene expression data after mapping:")
print(f"Shape before mapping: {gene_data.shape}")
print(f"Shape after mapping: {gene_expr_data.shape}")

# Preview the result
print("\nPreview of mapped gene expression data:")
print(gene_expr_data.head(3))

# Update gene_data variable to use the mapped expression data
gene_data = gene_expr_data

# Check for any problematic gene names (empty, too long, etc.)
problematic_genes = [gene for gene in gene_data.index if not gene or len(gene) > 20]
if problematic_genes:
    print(f"\nFound {len(problematic_genes)} problematic gene names")
    print(f"Examples: {problematic_genes[:5]}")
else:
    print("\nNo problematic gene names found")



Identifying mapping columns between datasets:
Example gene ID from expression data: 100009676_at
Expression data contains 19070 gene IDs
Example ID from annotation: 1_at
Annotation data contains 591200 entries
Number of gene IDs in expression data that match annotation IDs: 19070

Gene mapping dataframe created
Gene mapping shape: (19037, 2)
First few rows of gene mapping:
         ID                                               Gene
0      1_at                             alpha-1-B glycoprotein
1     10_at  N-acetyltransferase 2 (arylamine N-acetyltrans...
2    100_at                                adenosine deaminase
3   1000_at          cadherin 2, type 1, N-cadherin (neuronal)
4  10000_at  v-akt murine thymoma viral oncogene homolog 3 ...

Gene expression data after mapping:
Shape before mapping: (19070, 30)
Shape after mapping: (2034, 30)

Preview of mapped gene expression data:
      GSM3398477  GSM3398478  GSM3398479  GSM3398480  GSM3398481  GSM3398482  \
Gene                 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Create a clinical dataframe with sample IDs from gene expression data
sample_ids = normalized_gene_data.columns.tolist()
print(f"\nFound {len(sample_ids)} samples in gene expression data")

# From the background information, create a clinical dataframe with appropriate grouping
# The study has 10 samples per group (control, SCZ, BD(-), BD(+))
clinical_df = pd.DataFrame(index=sample_ids)

# Based on the study design, assign the first 10 samples to control, next 10 to SCZ
# This matches the schizophrenia trait we're analyzing
clinical_df[trait] = 0  # Default to control
if len(sample_ids) >= 20:  # Ensure we have enough samples
    clinical_df.loc[sample_ids[10:20], trait] = 1  # SCZ samples (samples 10-19)

print(f"Clinical dataframe shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# Transpose the gene data to have samples as rows
gene_data_t = normalized_gene_data.T

# Link the clinical and genetic data
linked_data = clinical_df.join(gene_data_t)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from Schizophrenia patients and controls, with data from left and right brain hemispheres."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Normalizing gene symbols...
Gene data shape before normalization: (2034, 30)
Gene data shape after normalization: (1171, 30)


Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE120340.csv

Found 30 samples in gene expression data
Clinical dataframe shape: (30, 1)
Clinical data preview:
            Schizophrenia
GSM3398477              0
GSM3398478              0
GSM3398479              0
GSM3398480              0
GSM3398481              0
Linked data shape: (30, 1172)
Linked data preview (first 5 rows, 5 columns):
            Schizophrenia      A1BG    A4GALT       AAA1     ABCC11
GSM3398477              0  5.688718  8.525499  48.138985  89.126400
GSM3398478              0  4.993095  8.285332  47.555330  90.615588
GSM3398479              0  5.121468  8.502409  46.579863  90.608181
GSM3398480              0  5.686842  8.447090  47.325990  90.314839
GSM3398481              0  5.564686  8.743342  48.065375  88.717268


Data shape after handling missing values: (30, 1172)
For the feature 'Schizophrenia', the least common label is '1' with 10 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Schizophrenia' in this dataset is fine.

Data shape after removing biased features: (30, 1172)


Linked data saved to ../../output/preprocess/Schizophrenia/GSE120340.csv
